<a href="https://colab.research.google.com/github/pembatt/Depression_Screening_Tool/blob/main/Screening_tool_USC_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Only thing to change should be the transcript itseld and the name when saving. Just put in new transcript and run all, at the end the number used should be there, dont forget to save

In [2]:
#Install and import all the necessary packages

#!pip install nltk # Already available in Colab
#!pip install spacy # Already available in Colab
#!pip install vaderSentiment
# !pip install openpyxl # Already available in Colab
# !pip install TensorFlow # Already available in Colab
# !pip install NumPy # Already available in Colab



import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from spacy import displacy

#mount Google Drive
#from google.colab import drive
#drive.mount('/content/drive')


# Load the dataset
#dataset = pd.read_excel(r"/content/drive/MyDrive/Project_datasets/dataset_1.xlsx")



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Processing the transcript

In [4]:
# dataset = pd.read_csv(r"/content/drive/MyDrive/Project_datasets/screening_tool_save.csv")

# If dataset has been uploaded to VM
dataset = pd.read_csv(r"/content/drive/MyDrive/Project_datasets/USC_dataset/300_TRANSCRIPT.csv")



In [5]:
type(dataset), dataset.shape

(pandas.core.frame.DataFrame, (174, 1))

In [6]:
dataset[:3]

,start_time\tstop_time\tspeaker\tvalue
0,36.588\t39.668\tEllie\thi i'm ellie thanks for...
1,39.888\t43.378\tEllie\ti was created to talk t...
2,43.728\t48.498\tEllie\tthink of me as a friend...


In [7]:
df = dataset

In [8]:
#It gives you a ugly column name by default change it to text for simplicity
column_name = df.columns.tolist()
column_name[0] = 'text'
df.columns = column_name

In [9]:
#Remove rows with Ellie's line

df = df[~df['text'].str.contains('Ellie')]

# Display the resulting DataFrame
df[:2]



,text
6,62.328\t63.178\tParticipant\tgood
9,68.978\t70.288\tParticipant\tatlanta georgia


In [10]:
import re
# Remove all numbers from each row
df = df.replace('\d+', '', regex=True)



In [11]:

# Remove the pattern from each row
df = df.replace('.\t.\tParticipant\t', '', regex=True)

In [12]:
#Check weather it's actually removed or not
df[:2]

,text
6,good
9,atlanta georgia


# Transcript is now simply plain text with all the extra gunk removed, now the transformers can be ran


# This section is for using models from hugging dace to add sentiment and analysis to text. Ignore if using processed data.

In [13]:
#install dependencies
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.8 MB/s eta 0:00:00


In [14]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model='distilbert-base-uncased-finetuned-sst-2-english')

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [15]:
# trying sentiment analysis on our whole df and appending it
# Note: this takes 8 minutes to process the whole df

df['new_sentiment'] = df['text'].apply(sentiment_pipeline)

df.shape

(87, 2)

So that label field on the original dataset (besides being numeric) is something else entirely.

In [16]:
#emotion analysis
emotion_pipeline = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True, truncation=True, max_length=512)


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [17]:
#trying emotion analysis on our whole df and appending it


df['emotion_analysis'] = df['text'].apply(emotion_pipeline)


In [18]:
df[:1]

,text,new_sentiment,emotion_analysis
6,good,"[{'label': 'POSITIVE', 'score': 0.999816119670...","[[{'label': 'anger', 'score': 0.02631921134889..."


In [20]:
#Trying a transformer model to see how it differs from the label


from transformers import pipeline
depression_classifier = pipeline("text-classification", model="paulagarciaserrano/roberta-depression-detection")


In [ ]:

depression_classifier('Iam happy  at the same time')


[{'label': 'not depression', 'score': 0.9972456693649292}]

In [21]:
df['text_classification'] = df['text'].apply(depression_classifier)


In [24]:
df[:1]

,text,new_sentiment,emotion_analysis,text_classification
6,good,"[{'label': 'POSITIVE', 'score': 0.999816119670...","[[{'label': 'anger', 'score': 0.02631921134889...","[{'label': 'not depression', 'score': 0.996175..."


# Clean Attributes added by transformers

First to be cleaned is Sentiment

In [27]:
# Code the sentiment values onto the dataset as separate columns
import ast

df['sentiment_label'] = [ast.literal_eval(str(sent_dict))[0]['label'] for sent_dict in df.new_sentiment]

df['sentiment_score'] = [ast.literal_eval(str(sent_dict))[0]['score'] for sent_dict in df.new_sentiment]

df.shape

(87, 6)

In [28]:
df[:2]

,text,new_sentiment,emotion_analysis,text_classification,sentiment_label,sentiment_score
6,good,"[{'label': 'POSITIVE', 'score': 0.999816119670...","[[{'label': 'anger', 'score': 0.02631921134889...","[{'label': 'not depression', 'score': 0.996175...",POSITIVE,0.999816
9,atlanta georgia,"[{'label': 'POSITIVE', 'score': 0.996592819690...","[[{'label': 'anger', 'score': 0.01011795643717...","[{'label': 'not depression', 'score': 0.997535...",POSITIVE,0.996593


In [29]:
# First, ensure that all entries in the 'emotion_analysis' column are indeed dictionaries
df['emotion_analysis'] = df['emotion_analysis'].apply(lambda x: {d['label']:d['score'] for d in x[0]})

emotions_df = df['emotion_analysis'].apply(pd.Series)

# Rename the columns with the corresponding emoji
emotions_df.columns = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

# Join the new DataFrame with the original one
df = pd.concat([df, emotions_df], axis=1)

# Drop the original 'emotion_analysis' column
df = df.drop(columns=['emotion_analysis'])

In [30]:
# Pick max emotion for analysis

# Create a list of the emotion column names
emotion_columns = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

# Iterate over the rows
for index, row in df.iterrows():
    # Find the column name with the highest score
    max_emotion = max(emotion_columns, key=lambda x: row[x])

    # Assign the max_emotion value to the emotion column
    df.at[index, 'emotion'] = max_emotion




In [31]:
# Code the depression values onto dataset as separate columns

# Code the sentiment values onto the dataset as separate columns

df['depression_model_label'] = [ast.literal_eval(str(sent_dict))[0]['label'] for sent_dict in df.text_classification]

df['depression_score'] = [ast.literal_eval(str(sent_dict))[0]['score'] for sent_dict in df.text_classification]

df.shape

(87, 15)

In [32]:
# Since the old column was separated and added to the df, drop the redundant column
df = df.drop('new_sentiment', axis=1)
df = df.drop('text_classification', axis=1)

In [33]:
df[:2]

,text,sentiment_label,sentiment_score,anger,disgust,fear,joy,neutral,sadness,surprise,emotion,depression_model_label,depression_score
6,good,POSITIVE,0.999816,0.026319,0.105510,0.004854,0.186471,0.632514,0.034914,0.009418,neutral,not depression,0.996175
9,atlanta georgia,POSITIVE,0.996593,0.010118,0.005205,0.008232,0.046166,0.844113,0.026578,0.059588,neutral,not depression,0.997536


# Analyzing the dataset for patterns

# Getting data for excel sheet

Chi square

In [34]:
#!pip install allennlp
!pip install emoji
import re, nltk, emoji, torch, nltk
import numpy as np
from numpy import asarray, zeros
from collections import Counter
nltk.download('stopwords')
nltk.download('punkt')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.6/356.6 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.6.0-py2.py3-none-any.whl size=351311 sha256=e4ec8006be7f5a811164a86ac614e7c0b09332ea86d46abca96bc33066a8afc4
  Stored in directory: /root/.cache/pip/wheels/ea/0b/64/114bc939d0083621aa41521e21be246c888260b8aa21e6c1ad
Successfully built emoji


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [35]:
#Counting the use of emojis
#!pip install emoji

def char_is_emoji(character):
    #en_emoji = emoji.UNICODE_EMOJI.get("en")
    #return character[0] in emoji.UNICODE_EMOJI_ENGLISH
    return emoji.is_emoji(character[0])

# Define a function to count emojis in a text
def count_emojis(text):
    emoji_count = 0
    for character in text:
        if char_is_emoji(character):
            emoji_count += 1
    return emoji_count

# Apply the function to the "text" column to count emojis
df['emoji_count'] = df['text'].apply(count_emojis)



print(df[['emoji_count']])


     emoji_count
6              0
9              0
12             0
14             0
16             0
..           ...
165            0
166            0
168            0
171            0
173            0

[87 rows x 1 columns]


In [22]:
#Counting emoticons

emoticons = ["[: )", ":-]", ":-3", ":->", "8-)", ":-}", ":)", ":]", ":3", ":>", "8)", ":}", ":o)", ":c)", ":^)", "=]", "=)", ":-))", ": D", "8 D", "x D", "X D", ":D", "8D", "xD", "XD", "=D", "=3", "B^D", ":-*", ":*", ":×'", "; )", "*-)", "; ]", ";)", "*)", ";]", ";^)", ": ,", ";D", "<3", "(:", ": (", ": c", ": <", ": [", ":(", ":c", ":<", ":[", ":-||", ">:[", ":{", ":@", ">:(", ":' (", ":'(", "D ':", "D:<", "D:", "D8", "D;", "D=", "DX", ": O", ": o", ":O", ":o", ":-0", "8 0", ">:O"]

# Define a function to count the usage of emoticons in a text
def count_emoticons(text):
    emoticon_count = sum(text.count(emoticon) for emoticon in emoticons)
    return emoticon_count





In [36]:
#adding the total socre for usage of  (! , emoticons , emojis, ?)
#creating the function

import emoji
# Sample DataFrame
#df_zero = pd.DataFrame({'text': ['Hello! How are you? 😃', 'I am feeling great! :D', 'No problem.']})

# Emoticons list
emoticons = ["[: )", ":-]", ":-3", ":->", "8-)", ":-}", ":)", ":]", ":3", ":>", "8)", ":}", ":o)", ":c)", ":^)", "=]", "=)", ":-))", ": D", "8 D", "x D", "X D", ":D", "8D", "xD", "XD", "=D", "=3", "B^D", ":-*", ":*", ":×'", "; )", "*-)", "; ]", ";)", "*)", ";]", ";^)", ": ,", ";D", "<3", "(:", ": (", ": c", ": <", ": [", ":(", ":c", ":<", ":[", ":-||", ">:[", ":{", ":@", ">:(", ":' (", ":'(", "D ':", "D:<", "D:", "D8", "D;", "D=", "DX", ": O", ": o", ":O", ":o", ":-0", "8 0", ">:O"]

#Emojis list
# Unicode range for emojis
start = 0x1F600
end = 0x1F64F

# Create a list of emojis
emojis = [chr(code) for code in range(start, end + 1)]

# Print the list of emojis
print(emojis)

# Function to count specific items and assign a score
def count_and_score(row):
    count = 0

    # Count exclamation marks
    count += row['text'].count('!')

    # Count emoticons
    emoticon_count = sum(row['text'].count(emoticon) for emoticon in emoticons)
    count += emoticon_count

    # Count emojis
    #emoji_count = len(emoji.findall(row['text']))
    emoji_pattern = r"[\U0001F600-\U0001F64F]|[\U0001F300-\U0001F5FF]|[\U0001F680-\U0001F6FF]|[\U0001F1E0-\U0001F1FF]|[\U00002702-\U000027B0]|[\U000024C2-\U0001F251]"
    emoji_count = len(re.findall(emoji_pattern, row['text']))
    count += emoji_count

    # Count question marks
    count += row['text'].count('?')

    # Assign a score based on count
    if count == 0:
        score = 0
    elif count == 1:
        score = 1
    elif count == 2:
        score = 2
    elif count == 3:
        score = 3
    else:
        score = 4

    return score




['😀', '😁', '😂', '😃', '😄', '😅', '😆', '😇', '😈', '😉', '😊', '😋', '😌', '😍', '😎', '😏', '😐', '😑', '😒', '😓', '😔', '😕', '😖', '😗', '😘', '😙', '😚', '😛', '😜', '😝', '😞', '😟', '😠', '😡', '😢', '😣', '😤', '😥', '😦', '😧', '😨', '😩', '😪', '😫', '😬', '😭', '😮', '😯', '😰', '😱', '😲', '😳', '😴', '😵', '😶', '😷', '😸', '😹', '😺', '😻', '😼', '😽', '😾', '😿', '🙀', '🙁', '🙂', '🙃', '🙄', '🙅', '🙆', '🙇', '🙈', '🙉', '🙊', '🙋', '🙌', '🙍', '🙎', '🙏']


In [39]:
#Apply the fuction to get count
df['emo_count'] = df.apply(count_and_score, axis=1)

In [40]:
sentiment_counts = df['sentiment_label'].value_counts()
emotion_counts = df['emotion'].value_counts()
depression_counts = df['depression_model_label'].value_counts()
emo_counts = df['emo_count'].value_counts()
# Display the counts

print(sentiment_counts)

print(emotion_counts)

print(depression_counts)

print(emo_counts)

POSITIVE    46
NEGATIVE    41
Name: sentiment_label, dtype: int64
neutral     57
joy         17
sadness      6
anger        4
surprise     2
fear         1
Name: emotion, dtype: int64
not depression    86
moderate           1
Name: depression_model_label, dtype: int64
0    87
Name: emo_count, dtype: int64


In [41]:
df.to_csv('Processed_PNumber.csv', index=False)